In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms


In [2]:
class Network(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Network, self).__init__()
        self.L1 = nn.Linear(input_size, 50)
        self.L2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.L1(x))
        return self.L2(x)


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
batch_size = 64
input_size = 784
num_classes = 10
learning_rate = 1e-3
num_epochs = 2

In [5]:
train_dataset = datasets.MNIST(
    root="./dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="./dataset/", train=False, transform=transforms.ToTensor(), download=True
)

In [6]:
train_loader = DataLoader(dataset = train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle=True)

In [7]:
model = Network(input_size=input_size, num_classes=num_classes).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)
        data = data.reshape(data.shape[0], -1)

        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

In [39]:
def get_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            x = x.reshape(x.shape[0], -1)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    print(f"Accuracy : {(num_correct / num_samples)*100:.2f}%")

In [40]:
get_accuracy(test_loader, model)

Accuracy : 94.66%


### Saving and Loading.

In [41]:
torch.save(model.state_dict(), "mnistModel.pt")

In [42]:
p = Network(num_classes=num_classes, input_size=input_size)

In [43]:
p.load_state_dict(torch.load("./mnistModel.pt"))

<All keys matched successfully>

In [44]:
p.eval()

Network(
  (L1): Linear(in_features=784, out_features=50, bias=True)
  (L2): Linear(in_features=50, out_features=10, bias=True)
)